In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
from data_prepration import get_training_set_data

In [3]:
df = get_training_set_data(seed=22)

In [4]:
df.reset_index(inplace=True, names='idx')
df.drop(columns=['question_row_id', 'answer_row_id', 'question_id', 'answer_id'], inplace=True)
# df['label'] = df['label'].astype(str)
# df['idx'] = df['idx'].astype(str)

In [5]:
train, validation = train_test_split(df, test_size=0.2, random_state=22)

In [6]:
train

,idx,question,answer,label
1750,148454,Why are you doing this for free? What motivate...,"Ankush\nAs I said, for me, it was Alexey's mac...",1
2866,141290,Do you have separate materials or videos on se...,"I do not have anything like that, but feel fre...",1
2036,105325,How relevant is Terraform in a data engineerin...,Victoria\nSQL is definitely super important. I...,1
4268,135096,"In the project, if we use Spark for transforma...","I wouldn't look at train at all, to be honest....",0
2532,26268,Linux Mint works way better than Ubuntu. It is...,"Yeah. If you want to experiment with Linux, yo...",1
...,...,...,...,...
989,34197,How does one move from a junior data engineer ...,Ankush\nWhat we use is basically unit testing....,1
2527,114019,If Kubernetes can spin up containers when need...,"Yes, we are.",0
2952,138977,If I have Kafka running on Virtual Machine 1 a...,Alexey\nYes. That's the homework and the first...,1
356,94638,"Are data engineers expected to know Redshift, ...",Ankush\nIf you're talking specifically about i...,1


In [7]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [62]:
tokenizer(
    ['hello','hi','hello hi'], 
    ['another example', '', 'IDK'], 
    ['hey', 'you', 'idk'], 
    padding = True
)

{'input_ids': [[101, 7592, 102, 2178, 2742, 102, 0], [101, 7632, 102, 102, 0, 0, 0], [101, 7592, 7632, 102, 8909, 2243, 102]], 'token_type_ids': [[0, 0, 0, 1, 1, 1, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1]], 'labels': [[101, 4931, 102, 0], [101, 2017, 102, 0], [101, 8909, 2243, 102]]}

In [8]:
train_tokenized = tokenizer(
    list(train['question']),
    list(train['answer']),
    padding = True, truncation=True    
)

validation_tokenized = tokenizer(
    list(validation['question']),
    list(validation['answer']),
    padding = True, truncation=True    
)

In [9]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy") # ddecrease learning rate

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_tokenized, train['label']))
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_tokenized, validation['label']))

In [13]:
train_encodings = np.array(train_tokenized["input_ids"])
train_labels = tf.constant(train['label'])

In [59]:
# model.fit(train_dataset, epochs=10, validation_data=validation_dataset)
# labels = tf.convert_to_tensor(train['label'])
# model.fit(**train_tokenized, labels=labels)
model.fit(train_encodings, train_labels, epochs=2)

Epoch 1/2
  2/109 [..............................] - ETA: 1:33:36 - loss: 2.3863

In [42]:
# Load the model and tokenizer
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Prepare the data
train_encodings = tokenizer(["Hello, world!", "How are you?"], truncation=True, padding=True)
train_labels = tf.constant([1, 0])
train_encodings = np.array(train_encodings["input_ids"])

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(train_encodings, train_labels, epochs=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
1/1 [==============================] - 6s 6s/step - loss: 4.4525 - accuracy: 0.5000
Epoch 2/2
1/1 [==============================] - 0s 136ms/step - loss: 4.5617 - accuracy: 0.5000


In [14]:
batch = dict(train_tokenized)
labels = tf.convert_to_tensor(train['label'])
model.train_on_batch(batch, labels)

AttributeError: 'int' object has no attribute 'shape'